In [1]:
import pandas as pd
import math
from IPython.display import display, HTML

In [13]:
df_amcl = pd.read_csv("2022-11-19/kotak/2022-11-19-14-24-50-3kali-amcl-odom-filtered-amcl_pose.csv")
df_odom = pd.read_csv("2022-11-19/kotak/2022-11-19-14-24-50-3kali-amcl-odom-filtered-odom.csv")
df_ekf = pd.read_csv("2022-11-19/kotak/2022-11-19-14-24-50-3kali-amcl-odom-filtered-odometry-filtered.csv")
df_imu = pd.read_csv("2022-11-19/kotak/2022-11-19-14-24-50-3kali-amcl-odom-filtered-odometry-imu.csv")

In [2]:
df_amcl = pd.read_csv("2022-11-19/3titik/2022-11-19-14-37-15-3titik-amcl_pose.csv")
df_odom = pd.read_csv("2022-11-19/3titik/2022-11-19-14-37-15-3titik-odom.csv")
df_ekf = pd.read_csv("2022-11-19/3titik/2022-11-19-14-37-15-3titik-odometry-filtered.csv")
df_imu = pd.read_csv("2022-11-19/3titik/2022-11-19-14-37-15-3titik-odometry-imu.csv")

In [14]:
df_amcl = df_amcl[[".header.stamp.secs", ".pose.pose.position.x", ".pose.pose.position.y"]]
df_odom = df_odom[[".header.stamp.secs", ".pose.pose.position.x", ".pose.pose.position.y"]]
df_ekf = df_ekf[[".header.stamp.secs", ".pose.pose.position.x", ".pose.pose.position.y"]]
df_imu = df_imu[[".header.stamp.secs", ".pose.pose.position.x", ".pose.pose.position.y"]]

In [15]:
print(len(df_ekf), len(df_amcl), len(df_odom), len(df_imu))

7178 260 19148 7178


In [ ]:
df_amcl[".header.stamp.secs"] 

In [16]:
print("amcl: " + str(df_amcl.isna().sum().sum()))
print("odom: " + str(df_odom.isna().sum().sum()))
print("ekf: " + str(df_ekf.isna().sum().sum()))
print("ekf imu: " + str(df_imu.isna().sum().sum()))

amcl: 0
odom: 0
ekf: 0
ekf imu: 0


In [ ]:
# display(HTML(df_ekf.to_html()))

In [ ]:
df_amcl[".header.stamp.secs"][0]

In [17]:
# AMCL JADI POROS
df_ekf_clean = pd.DataFrame(columns=[".header.stamp.secs", ".pose.pose.position.x", ".pose.pose.position.y"])
df_amcl_clean = pd.DataFrame(columns=[".header.stamp.secs", ".pose.pose.position.x", ".pose.pose.position.y"])
df_odom_clean = pd.DataFrame(columns=[".header.stamp.secs", ".pose.pose.position.x", ".pose.pose.position.y"])
df_imu_clean = pd.DataFrame(columns=[".header.stamp.secs", ".pose.pose.position.x", ".pose.pose.position.y"])
prev = 0
for j, i in enumerate(df_amcl[".header.stamp.secs"]):
  if prev == i:
    continue
  df_amcl_clean_ = df_amcl.iloc[j].to_frame().T
  df_ekf_clean_ = (df_ekf[df_ekf[".header.stamp.secs"] == i])
  df_odom_clean_ = (df_odom[df_odom[".header.stamp.secs"] == i])
  df_imu_clean_ = (df_imu[df_imu[".header.stamp.secs"] == i])
  try:
    df_ekf_clean_ = df_ekf_clean_.reset_index().iloc[0].to_frame().T
    df_odom_clean_ = df_odom_clean_.reset_index().iloc[0].to_frame().T
    df_imu_clean_ = df_imu_clean_.reset_index().iloc[0].to_frame().T
  except:
    df_amcl_clean_ = pd.DataFrame(columns=[".header.stamp.secs", ".pose.pose.position.x", ".pose.pose.position.y"])
    pass
  prev = i
  df_ekf_clean = pd.concat([df_ekf_clean, df_ekf_clean_], ignore_index = True, axis=0)
  df_amcl_clean = pd.concat([df_amcl_clean, df_amcl_clean_], ignore_index = True, axis=0)
  df_odom_clean = pd.concat([df_odom_clean, df_odom_clean_], ignore_index = True, axis=0)
  df_imu_clean = pd.concat([df_imu_clean, df_imu_clean_], ignore_index = True, axis=0)

In [ ]:
# EKF JADI POROS
df_ekf_clean = pd.DataFrame(columns=[".header.stamp.secs", ".pose.pose.position.x", ".pose.pose.position.y"])
df_amcl_clean = pd.DataFrame(columns=[".header.stamp.secs", ".pose.pose.position.x", ".pose.pose.position.y"])
df_odom_clean = pd.DataFrame(columns=[".header.stamp.secs", ".pose.pose.position.x", ".pose.pose.position.y"])
z = []
prev = 0

for j, i in enumerate(df_ekf[".header.stamp.secs"]):
  if prev == i:
    continue
  df_ekf_clean_ = df_ekf.iloc[j].to_frame().T
  df_amcl_clean_ = (df_amcl[df_amcl[".header.stamp.secs"] == i])
  if not df_amcl_clean_.empty:
    df_amcl_clean_.reset_index().iloc[0].to_frame().T
  else: df_amcl_clean_.reset_index().T
  df_odom_clean_ = (df_odom[df_odom[".header.stamp.secs"] == i]).reset_index().iloc[0].to_frame().T
  prev = i
  df_ekf_clean = pd.concat([df_ekf_clean, df_ekf_clean_], ignore_index = True, axis=0)
  df_amcl_clean = pd.concat([df_amcl_clean, df_amcl_clean_], ignore_index = True, axis=0)
  df_odom_clean = pd.concat([df_odom_clean, df_odom_clean_], ignore_index = True, axis=0)

In [18]:
print(len(df_ekf_clean), len(df_amcl_clean), len(df_odom_clean), len(df_imu_clean))

166 166 166 166


In [19]:
df_odom_clean[".header.stamp.secs"] = df_odom_clean[".header.stamp.secs"].astype(int)
df_ekf_clean[".header.stamp.secs"] = df_ekf_clean[".header.stamp.secs"].astype(int)
df_amcl_clean[".header.stamp.secs"] = df_amcl_clean[".header.stamp.secs"].astype(int)
df_imu_clean[".header.stamp.secs"] = df_imu_clean[".header.stamp.secs"].astype(int)

In [20]:
df_odom_clean.drop(labels='index', axis=1, inplace=True)
df_ekf_clean.drop(labels='index', axis=1, inplace=True)
df_imu_clean.drop(labels='index', axis=1, inplace=True)
try:
    df_amcl_clean.drop(labels='index', axis=1, inplace=True)
except:
    pass

In [ ]:
display(HTML(df_amcl_clean.to_html()), HTML(df_odom_clean.to_html()),HTML(df_ekf_clean.to_html()))

In [21]:
# Ground truth lintasan persegi
x_max = 5.5
y_max = -2.9
kel = abs((2*x_max)) + abs((2*y_max))
# print(kel)

pjg_data = len(df_ekf_clean)
new_time = kel/pjg_data
# print(new_time)

x_truth = []
y_truth = []
point_x = 0
point_y = 0
while(point_x<x_max):
    x_truth.append(point_x)
    y_truth.append(point_y)
    point_x += new_time

x_truth[-1] = 5.5
y_truth[-1] = 0

x_max = x_truth[-1]
point_x =  x_max


while(y_max<point_y):
    point_y -= new_time
    if point_y<y_max:
        break
    y_truth.append(point_y)
    x_truth.append(x_max)

y_max = y_truth[-1]
point_y = y_max


while(0<point_x):
    point_x -= new_time
    if point_x<0:
        break
    x_truth.append(point_x)
    y_truth.append(point_y)

# x_truth[-1] = 0
# y_truth[-1] = -2.8

x_max = x_truth[-1]
point_x =  x_max


while(point_y<0):
    point_y += new_time
    if 0<point_y:
        break
    y_truth.append(point_y)
    x_truth.append(x_max)

x_truth.append(0)
y_truth.append(0)

# print(point_x, point_y)
# print(x_truth[-1], y_truth[-1])
print(len(x_truth), len(y_truth))

165 165


In [8]:
x_three_truth = [0, 7, 5]
y_three_truth = [0, 21, 27]

In [22]:
import plotly.graph_objects as go

# Transparan
layout = go.Layout(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    showlegend=False,
    autosize=False,
    # width=800,
    # height=2400,
    margin=go.layout.Margin(
        l=20,
        r=20,
        b=20,
        t=20,
        pad = 4
    )
)

fig = go.Figure(layout=layout)

# Add traces
fig.add_trace(go.Scatter(x=df_amcl_clean[".pose.pose.position.x"], y=df_amcl_clean[".pose.pose.position.y"],
                    mode='lines+markers',
                    name='AMCL'))
fig.add_trace(go.Scatter(x=df_ekf_clean[".pose.pose.position.x"], y=df_ekf_clean[".pose.pose.position.y"],
                    mode='lines+markers',
                    name='EKF'))
# fig.add_trace(go.Scatter(x=df_odom_clean[".pose.pose.position.x"], y=df_odom_clean[".pose.pose.position.y"],
#                     mode='lines+markers',
#                     name='ODOM UNFILTERED'))
# fig.add_trace(go.Scatter(x=df_imu_clean[".pose.pose.position.x"], y=df_imu_clean[".pose.pose.position.y"],
#                     mode='lines+markers',
#                     name='EKF ENCODER-IMU'))

# truth lintasan persegi
# fig.add_trace(go.Scatter(x=x_truth, y=y_truth,
#                     mode='lines+markers',
#                     name='GROUND TRUTH'))

# truth 3 titik
# fig.add_trace(go.Scatter(x=x_three_truth, y=y_three_truth,
#                     mode='markers',
#                     name='GOAL',
#                     marker_size=20,
#                     marker_symbol='star'
#                     ))

# Remove grid, zero line, axis number
fig.update_xaxes(
    # showgrid=False, 
    # zeroline=False, 
    # visible=False,
    zerolinecolor='red',
    gridcolor='LightPink',
    )
fig.update_yaxes(
    # showgrid=False, 
    # zeroline=False, 
    # visible=False,
    zerolinecolor='red',
    gridcolor='LightPink',
    )

fig.update_layout(
    # font_family="Courier New",
    font_color="red",
    # font_size=70,
    # title_font_family="Times New Roman",
    # title_font_color="red",
    # legend_title_font_color="green"
)

fig.show()

In [23]:
NE_amcl = []
NEk_amcl = 0
x_amcl = df_amcl_clean[".pose.pose.position.x"]
y_amcl = df_amcl_clean[".pose.pose.position.y"]

for i,j in enumerate(x_truth):
    ex = x_amcl[i]-j
    ey = y_amcl[i]-y_truth[i]
    NEk = math.sqrt(ex**2+ey**2)
    NEk_amcl = NEk
    NE_amcl.append(NEk_amcl)

# print("NE AMCL: ")
# print(NE_amcl)

ME_amcl = sum(NE_amcl)/len(df_amcl_clean)
print("ME AMCL: ")
print(ME_amcl)

NE_ME = []
for i,j in enumerate(NE_amcl):
    NE_ME.append((j-ME_amcl)**2)

MSE_amcl = sum(NE_ME)/len(df_amcl_clean)
print("MSE AMCL: ")
print(MSE_amcl)

ME AMCL: 
2.9231522819040765
MSE AMCL: 
2.5066688544954454


In [26]:
print(len(NE_amcl))

165


In [ ]:
NE_ekf = []
NEk_ekf = 0
x_ekf = df_ekf_clean[".pose.pose.position.x"]
y_ekf = df_ekf_clean[".pose.pose.position.y"]

for i,j in enumerate(x_truth):
    ex = x_ekf[i]-j
    ey = y_ekf[i]-y_truth[i]
    NEk = math.sqrt(ex**2+ey**2)
    NEk_ekf = NEk
    NE_ekf.append(NEk_ekf)

# print("NE ekf: ")
# print(NE_ekf)

ME_ekf = sum(NE_ekf)/len(df_ekf_clean)
print("ME ekf: ")
print(ME_ekf)

NE_ME = []
for i,j in enumerate(NE_ekf):
    NE_ME.append((j-ME_ekf)**2)

MSE_ekf = sum(NE_ME)/len(df_ekf_clean)
print("MSE ekf: ")
print(MSE_ekf)

In [ ]:
NE_odom = []
NEk_odom = 0
x_odom = df_odom_clean[".pose.pose.position.x"]
y_odom = df_odom_clean[".pose.pose.position.y"]

for i,j in enumerate(x_truth):
    ex = x_odom[i]-j
    ey = y_odom[i]-y_truth[i]
    NEk = math.sqrt(ex**2+ey**2)
    NEk_odom = NEk
    NE_odom.append(NEk_odom)

# print("NE odom: ")
# print(NE_odom)

ME_odom = sum(NE_odom)/len(df_odom_clean)
print("ME odom: ")
print(ME_odom)

NE_ME = []
for i,j in enumerate(NE_odom):
    NE_ME.append((j-ME_odom)**2)

MSE_odom = sum(NE_ME)/len(df_odom_clean)
print("MSE odom: ")
print(MSE_odom)

In [ ]:
NE_imu = []
NEk_imu = 0
x_imu = df_imu_clean[".pose.pose.position.x"]
y_imu = df_imu_clean[".pose.pose.position.y"]

for i,j in enumerate(x_truth):
    ex = x_imu[i]-j
    ey = y_imu[i]-y_truth[i]
    NEk = math.sqrt(ex**2+ey**2)
    NEk_imu = NEk
    NE_imu.append(NEk_imu)

# print("NE imu: ")
# print(NE_imu)

ME_imu = sum(NE_imu)/len(df_imu_clean)
print("ME imu: ")
print(ME_imu)

NE_ME = []
for i,j in enumerate(NE_imu):
    NE_ME.append((j-ME_imu)**2)

MSE_imu = sum(NE_ME)/len(df_imu_clean)
print("MSE imu: ")
print(MSE_imu)

In [15]:
# 3 TITIK AMCL
NE_amcl = []
NEk_amcl = 0

x_three_ = [0, 7, 5, 0, 7, 5, 0]
y_three_ = [0, 21, 27, 0, 21, 27, 0]

x_amcl_goal = [0.284, 6.9296, 5.1139, 0.41, 6.9957, 5.063, 0.332]
y_amcl_goal = [-0.203, 20.39, 26.91, 0.013, 20.94, 26.985, -0.114]

for i,j in enumerate(x_three_):
    ex = x_amcl_goal[i]-j
    ey = y_amcl_goal[i]-y_three_[i]
    NEk = math.sqrt(ex**2+ey**2)
    NEk_amcl = NEk
    NE_amcl.append(NEk_amcl)

# print("NE AMCL: ")
# print(NE_amcl)

ME_amcl = sum(NE_amcl)/len(df_amcl_clean)
# print("ME AMCL: ")
# print(ME_amcl)

NE_ME = []
for i,j in enumerate(NE_amcl):
    NE_ME.append((j-ME_amcl)**2)

MSE_amcl = sum(NE_ME)/len(df_amcl_clean)
print("MSE AMCL: ")
print(MSE_amcl)

MSE AMCL: 
0.0020960802540620316


In [16]:
# 3 TITIK AMCL
NE_ekf = []
NEk_ekf = 0

x_three_ = [0, 7, 5, 0, 7, 5, 0]
y_three_ = [0, 21, 27, 0, 21, 27, 0]

x_ekf_goal = [0.2812, 6.9345, 5.101 , 0.403, 7.01, 5.052, 0.34]
y_ekf_goal = [-0.1816, 20.595, 26.89, 0.014, 21.1, 26.969, -0.114]

for i,j in enumerate(x_three_):
    ex = x_ekf_goal[i]-j
    ey = y_ekf_goal[i]-y_three_[i]
    NEk = math.sqrt(ex**2+ey**2)
    NEk_ekf = NEk
    NE_ekf.append(NEk_ekf)

# print("NE ekf: ")
# print(NE_ekf)

ME_ekf = sum(NE_ekf)/len(df_ekf_clean)
# print("ME ekf: ")
# print(ME_ekf)

NE_ME = []
for i,j in enumerate(NE_ekf):
    NE_ME.append((j-ME_ekf)**2)

MSE_ekf = sum(NE_ME)/len(df_ekf_clean)
print("MSE ekf: ")
print(MSE_ekf)

MSE ekf: 
0.0015497600648218335


In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error

ekf_x = df_ekf_clean[".pose.pose.position.x"]
ekf_y = df_ekf_clean[".pose.pose.position.y"]
amcl_x = df_amcl_clean[".pose.pose.position.x"]
amcl_y = df_amcl_clean[".pose.pose.position.y"]

ekf_x = ekf_x.to_numpy()
ekf_y = ekf_y.to_numpy()
amcl_x = amcl_x.to_numpy()
amcl_y = amcl_y.to_numpy()

# for i,j in enumerate(ekf_x):
#     acc_x = abs((j-amcl_x[i])/j)*100
#     acc_y = abs((j-amcl_y[i])/j)*100
#     print(acc_x)
#     print("-"*20)
#     print(acc_y)

mse_x = mean_squared_error(ekf_x, amcl_x)
mse_y = mean_squared_error(ekf_y, amcl_y)

rmse_x = mean_squared_error(ekf_x, amcl_x, squared=False)
rmse_y = mean_squared_error(ekf_y, amcl_y, squared=False)